 Custom Loss Function

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [9]:
df = pd.read_csv('housing.csv')
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [10]:
df = pd.get_dummies(df, columns=['ocean_proximity'], drop_first=True)

In [13]:
data = df.dropna()
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,False,False,True,False
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,False,False,True,False
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,False,False,True,False
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,False,False,True,False
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,True,False,False,False
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,True,False,False,False
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,True,False,False,False
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,True,False,False,False


In [14]:
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [17]:
def weighted_mse(y_true, y_pred):
    weights = tf.where(y_true > 200000, 2.0, 1.0)
    return tf.reduce_mean(tf.square(y_true - y_pred) * weights)

In [18]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1)
])

c:\Users\loalo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.compile(optimizer='adam', loss=weighted_mse, metrics=['mse'])

In [20]:
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 100431921152.0000 - mse: 55549804544.0000
Epoch 2/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 101739249664.0000 - mse: 56198189056.0000
Epoch 3/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 102060351488.0000 - mse: 56272269312.0000
Epoch 4/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 101010857984.0000 - mse: 55759978496.0000
Epoch 5/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 101529305088.0000 - mse: 55880187904.0000
Epoch 6/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99685859328.0000 - mse: 55004626944.0000
Epoch 7/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 99967836160.0000 - mse: 55135649792.0000
Epoch 8/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 96155500544.0000 - mse: 53170499584.0000
Epoch 9/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98816917504.0000 - mse: 54476341248.0000
Epoch 10/10
511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97437704192.0000 - mse: 5

In [21]:
y_pred = model.predict(X_test).flatten()

128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [22]:
standard_mse = mean_squared_error(y_test, y_pred)
custom_mse = np.mean((y_test - y_pred)**2 * np.where(y_test > 200000, 2.0, 1.0))

In [25]:
print("\nLoss Comparison Table:")
print(f"{'Standard MSE':<20}: {standard_mse:.2f}")
print(f"{'Custom Weighted MSE':<20}: {custom_mse:.2f}")



Loss Comparison Table:
Standard MSE        : 54121509558.50
Custom Weighted MSE : 98519015905.09
